# Autoencoder


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.max_open_warning'] = 0

Fetch our tools:

In [ ]:
from lib.nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from lib.nnbench import NNBench
from lib.nnvis import NNVis

A tool to plot the transfer function

In [ ]:
def plot_ADC(net):
    x = np.arange(-0.125, 1.125, 0.001).reshape(-1,1)
    outs = net(x)
    fig, ax = plt.subplots()
    ax.plot(x, outs, label="bit")
    ax.set_xlabel('learnings')  # Add an x-label to the axes.
    ax.set_ylabel('output')  # Add a y-label to the axes.
    ax.set_title("ADC output")  # Add a title to the axes.
    #ax.set_yscale('log')
    ax.legend()  # Add a legend.        

# Experiment 1: Training to identity
First off, let's see how well a randomly-initialized affine layer can learn the identity transform

In [ ]:
net = Network()
net.extend(AffineLayer(16,16))

In [ ]:
bench = NNBench(net)
bench.checkpoint_net()

In [ ]:
bench.accept_source_of_truth(((v, v) for v in (np.random.randn(16) for i in range(1<<31))))
#bench.training_batch_cluster(2)

In [ ]:
vis = NNVis(bench)

In [ ]:
net.eta = 0.032
vis.plot_learning(1000, 16)

In [ ]:
bench.rollback_net() if False else bench.randomize_net()
rates = np.logspace(-4.5, -4, base=2, num=32)
#cube = bench.learn_loss_cube(1000, rates)
#vis.plot_loss_cube()

In [ ]:
bench.rollback_net()
learned_track = bench.learn_track(1000, 16)

In [ ]:
traja = bench.analyze_learning_track(learned_track)
vis.plot_trajectory(traja)

# Adopt a manifold
Let us choose a latent space for our input vectors, of
$$
y = a(x-b)^c \\
x \in [1,2) \\
a, c \in [1/2, 2] \\
b \in [-1,1]
$$

In [ ]:
def bend_gen():
    while True:
        a, c = np.exp2(2 * np.random.rand(2) - 1)
        b = 2 * np.random.rand() - 1
        #print(a, b, c)
        yield np.array([a * (x - b) ** c for x in np.arange(1,2,1/16)]) \
            + 0.0 * np.random.randn(16)

In [ ]:
def doublem(g):
    for x in g:
        yield x, x

In [ ]:
bench.accept_source_of_truth(doublem(bend_gen()))
bench.training_batch_cluster(2)

In [ ]:
it = bend_gen()
ins = [next(it) for i in range(16)]
fig, ax = plt.subplots()
_ = [ax.plot(ins[i], label=f"{i}") for i in range(len(ins))]
ax.set_xlabel('x')
ax.set_ylabel('y')
_ = ax.set_title("examples")
#ax.set_yscale('log')
#ax.legend()       

In [ ]:
bench.randomize_net()

In [ ]:
%%prun -l 30
net.eta = 0.001
vis.plot_learning(10000, 16)

In [ ]:
net.eta = 0.0005
vis.plot_learning(20000, 16)

In [ ]:
bench.rollback_net()
net.eta = 0.001
learned_track = bench.learn_track(100000, 16)

In [ ]:
traja = bench.analyze_learning_track(learned_track)
vis.plot_trajectory(traja)

In [ ]:
vis.knobs_plot_learning(1000, 16)

In [ ]:
bench.rollback_net() if True else bench.randomize_net()
rates = np.logspace(-7, -5.5, base=2, num=100)
cube = bench.learn_loss_cube(1000, rates)
vis.plot_loss_cube()

In [ ]:
net = Network()
net.extend(AffineLayer(8,8))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
nets.append(net)

# Manifold
We adopt the a

In [ ]:
%whos